# 整理json

In [1]:
import json


In [5]:
with open('data/ntu_corpus.json', 'r', encoding='utf8') as f:
    data = json.load(f)

In [9]:
atayal_data = list(filter(lambda x: '泰雅' in x['meta']['language'], data))

In [10]:
len(atayal_data)

1399

In [49]:
import re

results = {}

def free_contains_element_with_chinese(l):
    return len(list(filter(lambda x: '#c' in x,l))) > 0

def get_chinese(l):
    return list(filter(lambda x: '#c' in x, l))[0].lstrip('#c ')

# 去掉 ...(1.2)
def clean_sentence(s):
    s = re.sub("\.\.\.", "", s)
    s = re.sub("\(\d\.\d\)", "", s)
    s = s.strip()
    
    return s 

for data in atayal_data:
#     print(data['num'])
    
    if len(data['gloss']) == 0:
        continue
        
    sentence = ' '.join(map(lambda x: x[0], data['gloss']))
    sentence = clean_sentence(sentence)
    
    # 如果 results 已包含某個文檔名稱作為key
    if data['file'] in results:
        if 's_a_span' in data:
            ch = get_chinese(data['free'])
            results[data['file']]['data'][-1]['ab'] += f' {sentence}'
            results[data['file']]['data'][-1]['ch'] = ch
            results[data['file']]['data'].append(
                {
                    'ab': '',
                    'ch': ''
                }
            )
        else:
            results[data['file']]['data'][-1]['ab'] += f' {sentence}'
            
            
    # 第一次讀取某個文檔第一行時才會遇到
    else:
        if 's_a_span' in data:
            ch = get_chinese(data['free'])
            results[data['file']] = {
                'speaker': data['meta']['speaker'],
                'data': [
                    {
                        'ab': sentence,
                        'ch': ch
                    }
                ]
            }

            results[data['file']]['data'].append(
                {
                    'ab': '',
                    'ch': ''
                }
            )
        else:
            results[data['file']] = {
                'speaker': data['meta']['speaker'],
                'data': [
                    {
                        'ab': sentence,
                        'ch': ''
                    }
                ]
            }

In [52]:
for chapter_name in results:
    print(f'=== {chapter_name} ===')
    for s in results[chapter_name]['data']:
        print(s['ab'])
        print(s['ch'])        

=== story/Atayal_Mayrinax/AtaNr-proposal_Takun ===
panqu'an-ay=mu tikay ku gaga' nku ita' ka itaal i m-usa' i,_ t<um>ayi' ma-na-'aring ga,\ magalpu,_ .. su kaal ku ulaqi' la ga,\ ga,\
我來敘述一些關於我們泰雅人去娶親的習俗，是等小孩到了十歲大就要開始準備的。
 tuliq ku yava' ru,\ nku kanayril yaya' nku kanayril,\ tam-- .. yava' nku tamalung,_ yaya nku ulaqi' na tamalung,\ mapatisal la ru==,_
男孩的父母和女孩的父母會互相聊天。
 il-- kia ku ulaqi'=mu na kanayril maha ku,_ .. kinsaxa'-an kia ku ulaqi'=mu na mak--,_ .. maliku maha ini'=nha' tal-i su,_ su,_ piya' su kaal uwi.\ .. <L2MhuancenL2M>,_ i m-aska' la ga,_ i suqun-aw=ta' la ku maha ma-kaal ru,\
(一方)說我們有一個女孩，一方說我們有一個男孩，也不看他們的年齡，只要差不多，就可以結婚。
 vali=nha' si-kaal sku ulaqi' ku,_ ku t<um>ayi' han.\ nanu' yaku navakis ku s<um>pung ru,_ tuliq,\ m-usa' vay i s<um>atu' su kavahniq.\
他們不會告訴小孩婚嫁的事，是由老人家決定並開始去提親。
 ku== ulaqi' na tama-- yava' nku ulaqi' na tamalung ini' ras-i ku ulaqi' mi.\
男孩的父親(去提親時)不會帶著男孩。
 m-usa' ru== m-aki raniq ru,\ siri==q maha aruwa i k<um>alax ku sisiliq ga mavuinah,\ ini' 

In [55]:
results['story/Atayal_Mayrinax/AtaNr-proposal_Takun']['data'][-2]

{'ab': ' nanu yani la.\\', 'ch': '就這樣。'}

# 輸出成word

In [56]:
from docx import Document

In [62]:
for f in results:
    document = Document()
    document.add_paragraph(f"speaker: {results[f]['speaker']}")
    document.add_paragraph(f"")    
    
    for s in results[f]['data']:
        document.add_paragraph(f"A: {s['ab'].strip()}")
        document.add_paragraph(f"RA: none")
        document.add_paragraph(f"G: none")
        document.add_paragraph(f"M: {s['ch'.strip()]}")
        document.add_paragraph(f"RM: none")
        document.add_paragraph(f"")    

    name = f.split('/')[-1]
    document.save(f'{name}.docx')